In [1]:
# 0. Importar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# Configuraciones de visualización
sns.set()  # estilo Seaborn (opcional)
plt.rcParams["figure.figsize"] = (8, 5)


In [10]:
# 1. Cargar los datasets manualmente desde Parquet
df_train = pd.read_parquet("/home/carizac/documnetos/proyecto_nlp/train-00000-of-00001-40d9940488cb84b1.parquet")
df_test = pd.read_parquet("/home/carizac/documnetos/proyecto_nlp/test-00000-of-00001-9cc7bacfd2a73ac7.parquet")

# 2. Verificar estructura
print("Entrenamiento:", df_train.shape)
print("Prueba:", df_test.shape)
df_train.head()
df_test.head()


Entrenamiento: (176192, 2)
Prueba: (75510, 2)


,text,label
0,"parece familiar?. probablemente no, ya que se ...",3
1,"el hotel, espectacular. el hotel es muy bonito...",5
2,lugar bueno escondido. amabilidad profesionali...,4
3,situacion privilegiada. hotel con una situacio...,5
4,caminata y visita al cerro monserrate. buen lu...,5
